In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

### $\ell_1$ and $\ell_2$ regularization

In [4]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor or 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 3ms/step - loss: 1.6071 - accuracy: 0.8108 - val_loss: 0.7300 - val_accuracy: 0.8212
Epoch 2/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7180 - accuracy: 0.8272 - val_loss: 0.6799 - val_accuracy: 0.8452


In [6]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 3ms/step - loss: 1.6152 - accuracy: 0.8124 - val_loss: 0.7576 - val_accuracy: 0.8086
Epoch 2/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7215 - accuracy: 0.8238 - val_loss: 0.7179 - val_accuracy: 0.8234


### drop out

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5733 - accuracy: 0.8034 - val_loss: 0.3580 - val_accuracy: 0.8682
Epoch 2/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4217 - accuracy: 0.8456 - val_loss: 0.3389 - val_accuracy: 0.8716


### Alpha Dropout

In [9]:
tf.random.set_seed(42)
np.random.seed(42)

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6654 - accuracy: 0.7599 - val_loss: 0.5935 - val_accuracy: 0.8420
Epoch 2/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5608 - accuracy: 0.7934 - val_loss: 0.5607 - val_accuracy: 0.8398
Epoch 3/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5284 - accuracy: 0.8055 - val_loss: 0.4848 - val_accuracy: 0.8592
Epoch 4/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5074 - accuracy: 0.8122 - val_loss: 0.4556 - val_accuracy: 0.8620
Epoch 5/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4925 - accuracy: 0.8169 - val_loss: 0.4593 - val_accuracy: 0.8574
Epoch 6/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4875 - accuracy: 0.8182 - val_loss: 0.4784 - val_accuracy: 0.8588
Epoch 7/20
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4718 - accuracy: 0.8252 - val_loss: 0.4623 - val_accuracy:

In [11]:
model.evaluate(X_test_scaled, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.4403 - accuracy: 0.8690


[0.4402810037136078, 0.8690000176429749]

In [12]:
model.evaluate(X_train_scaled, y_train)

1719/1719 [==============================] - 2s 1ms/step - loss: 0.3277 - accuracy: 0.8892


[0.3277498185634613, 0.8892182111740112]

In [13]:
history = model.fit(X_train_scaled, y_train)

1719/1719 [==============================] - 5s 3ms/step - loss: 0.4164 - accuracy: 0.8461


### MC Dropout

In [14]:
tf.random.set_seed(42)
np.random.seed(42)

In [15]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [16]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [17]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.32, 0.  , 0.15, 0.  , 0.54]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.9 , 0.  , 0.1 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.47, 0.  , 0.  , 0.  , 0.52]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.13, 0.  , 0.85]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.23, 0.  , 0.1 , 0.  , 0.66]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.35, 0.  , 0.65]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.28, 0.  , 0.14, 0.  , 0.58]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.41, 0.  , 0.59]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.82, 0.  , 0.03, 0.  , 0.15]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.11, 0.  , 0.05, 0.  , 0.84]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.08, 0.  , 0.9 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 0.  , 0.5 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.86, 0.  , 0.03, 0.  , 0.11]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.41, 0.  , 0.06, 0.  , 0

In [18]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.23, 0.  , 0.1 , 0.  , 0.67]],
      dtype=float32)

In [19]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.28, 0.  , 0.16, 0.  , 0.29]],
      dtype=float32)

In [20]:
y_pred = np.argmax(y_proba, axis=1)

In [21]:
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy


0.8697

In [22]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [23]:
tf.random.set_seed(42)
np.random.seed(42)

In [24]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [25]:
mc_model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
mc_alpha_dropout (MCAlphaDro (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 300)               235500    
_________________________________________________________________
mc_alpha_dropout_1 (MCAlphaD (None, 300)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               30100     
_________________________________________________________________
mc_alpha_dropout_2 (MCAlphaD (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)               

In [26]:
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [27]:
mc_model.set_weights(model.get_weights())


Now we can use the model with MC Dropout:

In [28]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.24, 0.  , 0.17, 0.  , 0.6 ]],
      dtype=float32)

### Max norm

In [29]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

In [30]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4746 - accuracy: 0.8329 - val_loss: 0.3783 - val_accuracy: 0.8630
Epoch 2/2
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3546 - accuracy: 0.8715 - val_loss: 0.3612 - val_accuracy: 0.8704
